In [2]:

#Importing Selenium
from selenium import webdriver 
import selenium 
from selenium.webdriver.firefox.options import Options
import time 

options = Options()
options.headless = True

#Setting up webdriver client
browser = webdriver.Firefox(options = options)
browser.get("https://www.myodds.bet/")    
browser.maximize_window() 

In [3]:
#Finding each Match
search_box = browser.find_elements_by_css_selector('div.match-detail') 
#Closing a popup that upstructs element clicking later.
close_button = browser.find_elements_by_css_selector('.mat-raised-button')
close_button[0].click()

In [4]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
test_db = client.test_database
test_col = test_db.test_collection

In [ ]:
def scrape_basic_info(dic, game):
    dic['time'] = game[0]
    dic['lines'] = game[1][1:]
    dic['team_one'] = game[2] 
    dic['team_two'] = game[6]
    dic['event'] = {}

In [ ]:
def scrape_game_title(dic, game):
    img = game.find_elements_by_tag_name('img')
    for element in img:
        game_title = element.get_attribute('title')
        if game_title != '':
            dic['Game'] = element.get_attribute('title')

In [ ]:
#Iterating through each match
for box in search_box:
    game = box.text.splitlines()
    #Checking if the match have 7 text elements. This is used to filter out the matches without betting lines.
    #May have to set to >= 7, because the league or tournament may be included on this line.
    if len(game) == 7:
        dic = {} #Match Dictionary
        scrape_basic_info(dic = dic, game = game)
        scrape_game_title(dic = dic, game = box)
        #Clicking on the match to show all of the lines availbe. Named drips.
        box.click()   
        drips = box.find_elements_by_css_selector('div.detail > div:nth-child(n)')

        for i in range(len(drips)):
            if i == 0: #The first item corresponds to lines that relate to the whole match not an individual map.
                dic['event']['match'] = {}
                lines = drips[i].find_elements_by_css_selector(f'div.detail > div:nth-child({i + 1}) > div:nth-child(n)')
                for line in lines:
                    if len(line.text) >= 1: #Making sure the line has text. Some don't.
                        line.click()
                        split_line = line.text.splitlines()
                        if split_line[1][0].isdecimal():
                            #First text element corresponds to the actaul betting event (winner, handicap, first blood, etc.)
                            bet_line = split_line[0].replace('.', '_')
                            dic['event']['match'][bet_line] = {}
                            #The next two text elements are the most competitive lines for those events, this can be found out after scrapping.
                            split_line_clean = split_line[3:]
                            #Info comes in threes: Company, team_one_odd, team_two_odd respectively
                            for com, one, two in zip(split_line_clean[::3], split_line_clean[1::3], split_line_clean[2::3]):
                                com_clean = com.replace('.', '_')
                                dic['event']['match'][bet_line][com_clean] = {}
                                dic['event']['match'][bet_line][com_clean]['team_one_odd'] = float(one)
                                dic['event']['match'][bet_line][com_clean]['team_two_odd'] = float(two)
                                dic['event']['match'][bet_line][com_clean]['time'] = time.time()

            else:
                #Finding the map or event type.
                event_type = drips[i].text.splitlines()[0].lower().replace(' ','_').replace('.', '_')
                dic['event'][event_type] = {}
                #The f string {i + 1} has the +1 becuase the count starts at 1 not 0 like in python
                lines = drips[i].find_elements_by_css_selector(f'div.detail > div:nth-child({i + 1}) > div:nth-child(n)')
                for k in range(len(lines)):
                    if k != 0: #The first line corresponds to the map or event type. Redundant and easier to avoid in this loop
                        if len(lines[k].text) >= 1: #Making sure the line has text. Some don't.
                            lines[k].click() #Opening all of the lines by company
                            split_line = lines[k].text.splitlines()

                            if split_line[1][0].isdecimal():
                                #First text element corresponds to the actaul betting event (winner, handicap, first blood, etc.)
                                bet_line = split_line[0].replace('.', '_')
                                dic['event'][event_type][bet_line] = {}
                                #The next two text elements are the most competitive lines for those events, this can be found out after scrapping.
                                split_line_clean = split_line[3:]

                                #Info comes in threes: Company, team_one_odd, team_two_odd respectively
                                for com, one, two in zip(split_line_clean[::3], split_line_clean[1::3], split_line_clean[2::3]):
                                    com_clean = com.replace('.', '_')
                                    dic['event'][event_type][bet_line][com_clean] = {}
                                    dic['event'][event_type][bet_line][com_clean]['team_one_odd'] = float(one)
                                    dic['event'][event_type][bet_line][com_clean]['team_two_odd'] = float(two)
                                    dic['event'][event_type][bet_line][com_clean]['time'] = time.time()


        #This adding to database
        dic_id = test_col.insert_one(dic).inserted_id

In [5]:

#This list is just a place holder for a database. The match scrapped below will be put into a Mongo DB.


ticker = 0
#Iterating through each match
for box in search_box:
    game = box.text.splitlines()
    #Checking if the match have 7 text elements. This is used to filter out the matches without betting lines.
    #May have to set to >= 7, because the league or tournament may be included on this line.
    if len(game) == 7:
        dic = {} #Match Dictionary


        dic['time'] = game[0] #Time of the start of the match
        dic['lines'] = game[1][1:] #Amount of betting lines for the match
        dic['team_one'] = game[2] 
        #game[3] = The most competitive odds for team 1 to win the match
        #game[4] = 'vs'
        #game[5] = The most competitive odds for team 2 to win the match
        dic['team_two'] = game[6]
        dic['event'] = {} #Betting event dictionary

        #Extracting the images in the match and finding the one with a title.
        #This title is the type of game being played. (CSGO, DOTA, etc.)
        img = box.find_elements_by_tag_name('img')
        for element in img:
            game_title = element.get_attribute('title')
            if game_title != '':
                dic['Game'] = element.get_attribute('title')

        #Clicking on the match to show all of the lines availbe. Named drips.
        box.click()   
        drips = box.find_elements_by_css_selector('div.detail > div:nth-child(n)')

        for i in range(len(drips)):
            if i == 0: #The first item corresponds to lines that relate to the whole match not an individual map.
                dic['event']['match'] = {}
                lines = drips[i].find_elements_by_css_selector(f'div.detail > div:nth-child({i + 1}) > div:nth-child(n)')
                for line in lines:
                    if len(line.text) >= 1: #Making sure the line has text. Some don't.
                        line.click()
                        split_line = line.text.splitlines()
                        if split_line[1][0].isdecimal():
                            #First text element corresponds to the actaul betting event (winner, handicap, first blood, etc.)
                            bet_line = split_line[0].replace('.', '_')
                            dic['event']['match'][bet_line] = {}
                            #The next two text elements are the most competitive lines for those events, this can be found out after scrapping.
                            split_line_clean = split_line[3:]
                            #Info comes in threes: Company, team_one_odd, team_two_odd respectively
                            for com, one, two in zip(split_line_clean[::3], split_line_clean[1::3], split_line_clean[2::3]):
                                com_clean = com.replace('.', '_')
                                dic['event']['match'][bet_line][com_clean] = {}
                                dic['event']['match'][bet_line][com_clean]['team_one_odd'] = float(one)
                                dic['event']['match'][bet_line][com_clean]['team_two_odd'] = float(two)
                                dic['event']['match'][bet_line][com_clean]['time'] = time.time()

            else:
                #Finding the map or event type.
                event_type = drips[i].text.splitlines()[0].lower().replace(' ','_').replace('.', '_')
                dic['event'][event_type] = {}
                #The f string {i + 1} has the +1 becuase the count starts at 1 not 0 like in python
                lines = drips[i].find_elements_by_css_selector(f'div.detail > div:nth-child({i + 1}) > div:nth-child(n)')
                for k in range(len(lines)):
                    if k != 0: #The first line corresponds to the map or event type. Redundant and easier to avoid in this loop
                        if len(lines[k].text) >= 1: #Making sure the line has text. Some don't.
                            lines[k].click() #Opening all of the lines by company
                            split_line = lines[k].text.splitlines()

                            if split_line[1][0].isdecimal():
                                #First text element corresponds to the actaul betting event (winner, handicap, first blood, etc.)
                                bet_line = split_line[0].replace('.', '_')
                                dic['event'][event_type][bet_line] = {}
                                #The next two text elements are the most competitive lines for those events, this can be found out after scrapping.
                                split_line_clean = split_line[3:]

                                #Info comes in threes: Company, team_one_odd, team_two_odd respectively
                                for com, one, two in zip(split_line_clean[::3], split_line_clean[1::3], split_line_clean[2::3]):
                                    com_clean = com.replace('.', '_')
                                    dic['event'][event_type][bet_line][com_clean] = {}
                                    dic['event'][event_type][bet_line][com_clean]['team_one_odd'] = float(one)
                                    dic['event'][event_type][bet_line][com_clean]['team_two_odd'] = float(two)
                                    dic['event'][event_type][bet_line][com_clean]['time'] = time.time()


        #This adding to database
        dic_id = test_col.insert_one(dic).inserted_id
    ticker += 1
    print(ticker / len(search_box))



0.018867924528301886
0.03773584905660377
0.05660377358490566
0.07547169811320754
0.09433962264150944
0.11320754716981132
0.1320754716981132
0.1509433962264151
0.16981132075471697
0.18867924528301888
0.20754716981132076
0.22641509433962265
0.24528301886792453
0.2641509433962264
0.2830188679245283
0.3018867924528302
0.32075471698113206
0.33962264150943394
0.3584905660377358
0.37735849056603776
0.39622641509433965
0.41509433962264153
0.4339622641509434
0.4528301886792453
0.4716981132075472
0.49056603773584906
0.5094339622641509
0.5283018867924528
0.5471698113207547
0.5660377358490566
0.5849056603773585
0.6037735849056604
0.6226415094339622
0.6415094339622641
0.660377358490566
0.6792452830188679
0.6981132075471698
0.7169811320754716
0.7358490566037735
0.7547169811320755
0.7735849056603774
0.7924528301886793
0.8113207547169812
0.8301886792452831
0.8490566037735849
0.8679245283018868
0.8867924528301887
0.9056603773584906
0.9245283018867925
0.9433962264150944
0.9622641509433962
0.981132075471

In [6]:
# .detail > div:nth-child(1) > div:nth-child(1)
# div.ng-tns-c18-15
# print(m_list[0]['event']['map 2']['Winner']['LOOT.BET'])


In [7]:
import pprint
pprint.pprint(test_col.find_one())

{'Game': 'CSGO',
 '_id': ObjectId('5d797a2842ec4a9e6416fb91'),
 'event': {'map_1': {},
           'map_2': {},
           'map_3': {},
           'match': {'Winner': {'LOOT_BET': {'team_one_odd': 1.35,
                                             'team_two_odd': 3.05,
                                             'time': 1568242215.2403622}}}},
 'lines': '1',
 'team_one': 'MenaceGG',
 'team_two': 'Feniks eSports',
 'time': 'Live'}


In [8]:
test_col.count_documents({})

92

In [43]:
# from bson.son import SON
for doc in test_col.find({'Game':'CSGO'}):
    print(doc)

{'_id': ObjectId('5d797a2842ec4a9e6416fb91'), 'time': 'Live', 'lines': '1', 'team_one': 'MenaceGG', 'team_two': 'Feniks eSports', 'event': {'match': {'Winner': {'LOOT_BET': {'team_one_odd': 1.35, 'team_two_odd': 3.05, 'time': 1568242215.2403622}}}, 'map_1': {}, 'map_2': {}, 'map_3': {}}, 'Game': 'CSGO'}
{'_id': ObjectId('5d797a2942ec4a9e6416fb92'), 'time': 'Live', 'lines': '1', 'team_one': 'AVEZ', 'team_two': 'wine and rain', 'event': {'match': {'Winner': {'LOOT_BET': {'team_one_odd': 1.25, 'team_two_odd': 3.75, 'time': 1568242217.1579108}}}}, 'Game': 'CSGO'}
{'_id': ObjectId('5d797a2b42ec4a9e6416fb94'), 'time': 'Live', 'lines': '1', 'team_one': 'Sons of Lennart', 'team_two': 'Granit Gaming', 'event': {'match': {'Winner': {'LOOT_BET': {'team_one_odd': 1.44, 'team_two_odd': 2.65, 'time': 1568242219.1919923}}}}, 'Game': 'CSGO'}
{'_id': ObjectId('5d797a2c42ec4a9e6416fb95'), 'time': 'Live', 'lines': '1', 'team_one': 'EsportsAdviser', 'team_two': 'visomvet', 'event': {'match': {'Winner': {'

In [63]:
df = pd.DataFrame(columns = ['Game', 'team_one', 'team_two', 'company', 'team_one_odd', 'team_two_odd', 'time'])
for doc in test_col.find():
    for com in doc['event']['match']['Winner']:
        df = df.append(pd.Series([
        doc['Game'],
        doc['team_one'],
        doc['team_two'],
        com,
        doc['event']['match']['Winner'][com]['team_one_odd'],
        doc['event']['match']['Winner'][com]['team_two_odd'],
        doc['event']['match']['Winner'][com]['time']
        ], index= df.columns), ignore_index=True)
df.head()
        

    
#     print(pd.DataFrame(doc['event']['match']['Winner']))
#     print(f"Odds of {doc['team_one']} vs {doc['team_two']} in {doc['Game']}")

,Game,team_one,team_two,company,team_one_odd,team_two_odd,time
0,CSGO,MenaceGG,Feniks eSports,LOOT_BET,1.35,3.05,1.568242e+09
1,CSGO,AVEZ,wine and rain,LOOT_BET,1.25,3.75,1.568242e+09
2,LOL,Excel Esports,asus rog elite,LOOT_BET,1.06,8.00,1.568242e+09
3,CSGO,Sons of Lennart,Granit Gaming,LOOT_BET,1.44,2.65,1.568242e+09
4,CSGO,EsportsAdviser,visomvet,LOOT_BET,2.48,1.50,1.568242e+09


In [85]:
no_time = df.loc[:,['Game', 'team_one', 'team_two', 'company']]
dupper = no_time[no_time.duplicated()]
# dup = df[df['Game', 'team_one', 'team_two', 'company', 'team_one_odd', 'team_two_odd'].duplicated()]
len(df)


446

In [246]:
clean_df = df.drop_duplicates(subset = ['Game', 'team_one', 'team_two', 'company'])
len(clean_df)

435

/home/sean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Game,team_one,team_two,company,team_one_odd,team_two_odd,time
15,CSGO,Sharks Esports Team,Mythic,ArcaneBet,1.00,13.96,1.568246e+09
16,CSGO,Sharks Esports Team,Mythic,Buff_bet,1.00,14.03,1.568246e+09
17,CSGO,Sharks Esports Team,Mythic,GG_BET,1.00,14.29,1.568246e+09
18,CSGO,Sharks Esports Team,Mythic,LOOT_BET,1.01,11.10,1.568246e+09
19,CSGO,Sharks Esports Team,Mythic,Rivalry,1.07,7.50,1.568246e+09
408,CSGO,Sharks Esports Team,team divine,10bet,1.31,3.20,1.568274e+09
409,CSGO,Sharks Esports Team,team divine,ArcaneBet,1.38,2.87,1.568274e+09
410,CSGO,Sharks Esports Team,team divine,Bethard,1.31,3.20,1.568274e+09
411,CSGO,Sharks Esports Team,team divine,Buff_bet,1.38,2.87,1.568274e+09


In [92]:
for ga in clean_df.Game.unique():
    print(ga)
    print(len(clean_df[df['Game'] == ga]))

CSGO
358
LOL
16
DOTA
25
SC2
36


/home/sean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: too many values to unpack (expected 1)

In [110]:
clean_df.loc[(clean_df['team_one'] == 'Sharks Esports Team') & (clean_df['team_two'] == 'Mythic')].head()

,Game,team_one,team_two,company,team_one_odd,team_two_odd,time
15,CSGO,Sharks Esports Team,Mythic,ArcaneBet,1.00,13.96,1.568246e+09
16,CSGO,Sharks Esports Team,Mythic,Buff_bet,1.00,14.03,1.568246e+09
17,CSGO,Sharks Esports Team,Mythic,GG_BET,1.00,14.29,1.568246e+09
18,CSGO,Sharks Esports Team,Mythic,LOOT_BET,1.01,11.10,1.568246e+09
19,CSGO,Sharks Esports Team,Mythic,Rivalry,1.07,7.50,1.568246e+09


,Game,team_one,team_two,company,team_one_odd,team_two_odd,time
0,CSGO,MenaceGG,Feniks eSports,LOOT_BET,1.35,3.05,1.568242e+09
1,CSGO,AVEZ,wine and rain,LOOT_BET,1.25,3.75,1.568242e+09
2,LOL,Excel Esports,asus rog elite,LOOT_BET,1.06,8.00,1.568242e+09
3,CSGO,Sons of Lennart,Granit Gaming,LOOT_BET,1.44,2.65,1.568242e+09
4,CSGO,EsportsAdviser,visomvet,LOOT_BET,2.48,1.50,1.568242e+09
5,CSGO,SJ Gaming,Planetkey Dynamics,LOOT_BET,1.17,4.70,1.568242e+09
6,LOL,Vitality Bee,Origen BCN,LOOT_BET,1.42,2.75,1.568242e+09
7,CSGO,MAD Lions,eXploit Esports,LOOT_BET,1.35,3.05,1.568242e+09
8,CSGO,Lilmix,Wind and Rain Sweden,LOOT_BET,1.65,2.15,1.568242e+09
10,CSGO,eXploit Esports,pRoNetAgEm,LOOT_BET,1.40,2.80,1.568242e+09


In [247]:
# clean_df.drop(clean_df.loc[(clean_df['team_one_odd'] == 1.00) | (clean_df['team_two_odd'] == 1.00)])
clean_df = clean_df.loc[lambda df: (clean_df['team_one_odd'] != 1.00) & (clean_df['team_two_odd'] != 1.00)]
len(clean_df)

432

In [248]:
clean_df['margin'] = 1/clean_df['team_one_odd'] + 1/clean_df['team_two_odd']
clean_df['true_one_odd'] = clean_df['margin'] * clean_df['team_one_odd']
clean_df['true_two_odd'] = clean_df['margin'] * clean_df['team_two_odd']
clean_df.head()

,Game,team_one,team_two,company,team_one_odd,team_two_odd,time,margin,true_one_odd,true_two_odd
0,CSGO,MenaceGG,Feniks eSports,LOOT_BET,1.35,3.05,1.568242e+09,1.068610,1.442623,3.259259
1,CSGO,AVEZ,wine and rain,LOOT_BET,1.25,3.75,1.568242e+09,1.066667,1.333333,4.000000
2,LOL,Excel Esports,asus rog elite,LOOT_BET,1.06,8.00,1.568242e+09,1.068396,1.132500,8.547170
3,CSGO,Sons of Lennart,Granit Gaming,LOOT_BET,1.44,2.65,1.568242e+09,1.071803,1.543396,2.840278
4,CSGO,EsportsAdviser,visomvet,LOOT_BET,2.48,1.50,1.568242e+09,1.069892,2.653333,1.604839


In [249]:

clean_df = clean_df[clean_df['margin'] >= 1]
len(clean_df)

428

In [250]:
grouped = clean_df.groupby(['team_one', 'team_two'])[['team_one_odd', 'team_two_odd', 'time']].max()
grouped['market_margin'] = 1 / grouped['team_one_odd'] + 1 / grouped['team_two_odd']
grouped['percent_return'] = 100 * (1/grouped['market_margin'] - 1)
grouped['time'] = pd.to_datetime(grouped['time'], unit='s')
grouped.sort_values(['market_margin']).head(20)

,,team_one_odd,team_two_odd,time,market_margin,percent_return
team_one,team_two,,,,,
swole patrol,Bushido Boyz,1.94,3.93,2019-09-12 00:20:46.317218065,0.769917,29.884157
Peeker's Advantage,The Quest,5.05,1.47,2019-09-12 00:21:46.953475237,0.878292,13.857362
Big Frames,LiViD Gaming,2.20,2.25,2019-09-12 00:22:02.261606693,0.898990,11.235955
team miracle,virgin gaming,2.20,2.13,2019-09-12 07:28:02.206386328,0.924029,8.221709
GuMiho,KeeN,1.55,3.25,2019-09-12 07:38:52.842597961,0.952854,4.947917
Team One,ex-Variance Gaming,1.32,4.40,2019-09-12 00:21:29.846235991,0.984848,1.538462
Vitality Bee,BIG,1.77,2.38,2019-09-12 07:34:07.964322805,0.985140,1.508434
Invictus Gaming,august,1.97,2.08,2019-09-12 07:28:40.525511503,0.988383,1.175309
Team Adroit,KG.Luminous,1.50,3.05,2019-09-12 00:23:41.804754019,0.994536,0.549451


In [301]:
clean_df[clean_df['team_one'] == 'Origen BCN']

,Game,team_one,team_two,company,team_one_odd,team_two_odd,time,margin,true_one_odd,true_two_odd,team_one_win,true_one_per,imp_per
346,LOL,Origen BCN,Rogue,10bet,1.61,2.00,1.568274e+09,1.121118,1.805000,2.242236,False,0.554017,0.621118
347,LOL,Origen BCN,Rogue,ArcaneBet,1.83,1.84,1.568274e+09,1.089926,1.994565,2.005464,False,0.501362,0.546448
348,LOL,Origen BCN,Rogue,Bethard,1.61,2.00,1.568274e+09,1.121118,1.805000,2.242236,False,0.554017,0.621118
349,LOL,Origen BCN,Rogue,Betway,1.60,2.20,1.568274e+09,1.079545,1.727273,2.375000,False,0.578947,0.625000
350,LOL,Origen BCN,Rogue,Buff_bet,1.84,1.86,1.568274e+09,1.081113,1.989247,2.010870,False,0.502703,0.543478
351,LOL,Origen BCN,Rogue,GG_BET,1.71,2.08,1.568274e+09,1.065565,1.822115,2.216374,False,0.548813,0.584795
352,LOL,Origen BCN,Rogue,LOOT_BET,1.64,2.17,1.568274e+09,1.070586,1.755760,2.323171,False,0.569554,0.609756


In [293]:
group_sort = grouped.sort_values(['market_margin'])
winners = group_sort.iloc[[4, 6, 9, 10, 11]]
# (winners['percent_return'] / 100 + 1).prod()
winners

,,team_one_odd,team_two_odd,time,market_margin,percent_return
team_one,team_two,,,,,
GuMiho,KeeN,1.55,3.25,2019-09-12 07:38:52.842597961,0.952854,4.947917
Vitality Bee,BIG,1.77,2.38,2019-09-12 07:34:07.964322805,0.985140,1.508434
eUnited,Team Envy,1.47,3.18,2019-09-12 07:35:53.088880062,0.994738,0.529032
Team Spirit,CR4ZY,4.50,1.29,2019-09-12 07:34:16.475241423,0.997416,0.259067
Origen BCN,Rogue,1.84,2.20,2019-09-12 07:34:44.631396770,0.998024,0.198020


In [255]:
results = ['swole patrol', 'Bushido Boyz', 1, "Peeker's Advantage", 'The Quest', 1, 'Big Frames', 'LiViD Gaming', 1, 'team miracle', 'virgin gaming', True, 'GuMiho', 'KeeN', True, 'Vitality Bee', 'BIG', False, 'Invictus Gaming', 'august', True, 'Team Adroit', 'KG.Luminous', False, 'eUnited', 'Team Envy', False, 'Team Spirit', 'CR4ZY', True, 'Origen BCN', 'Rogue', False, 'imperial e-sports', 'Redemption eSports POA', 1, 'swole patrol', 'Team Envy', False, 'Typhoon E-Sports Club', 'Resurgence', True, 'INTZ Academy', 'uppercut', False, 'madlikewizards', 'Downfall Gaming', True, 'Team Reapers', 'Skullz Gaming', True, 'Genuine Gaming', 'Chiefs Esports Club', True, 'Dream', 'Patience', True, 'divine', 'Stay Frosty', True, 'BIG', 'Team GamerLegion', True, 'Sharks Esports Team', 'Mythic', True, 'eUnited', 'Sharks Esports Team', True, 'Se7en eSports', 'BLUEJAYS', True, 'Avant Gaming', 'Jade', True, 'mousesports', 'CR4ZY', True, 'Paradox Gaming', 'parallax', True, 'BPro Gaming', 'ex-Epsilon Esports', False, 'forze', 'Sprout', True, 'LiViD Gaming', 'The Quest', False, 'Heroic', 'Tricked Esport', False, 'Nerchio', 'TLO', True, 'fury', 'Chiefs Esports Club', 1, 'Sharks Esports Team', 'team divine', True, 'Team Maple', 'arkangel', False, 'Team Heretics', 'Ambush Esport', True, 'Aristocracy', 'SMASH Esports', True, 'ATK', 'Monstars', True, 'Team One', 'Party Astronauts', True, 'black dragons female', 'Evidence e-Sports', False, 'Bushido Boyz', 'Rap Gang', False, 'New Identity', 'Mythic', True, 'Tricked Esports', 'Heroic', False, 'King Panda Gaming', 'abyss', True, 'Riot Squad Esports', 'The Quest', True, 'Windigo Gaming', 'Espada', True, 'ATK', 'Team One', True, 'Japaleno', 'M1X', False, 'Big Time Regal Gaming', 'Lynn Vision Gaming', True, 'Grayhound Gaming', 'Avant Gaming', True, 'team divine', 'Sharks Esports Team', False, 'Mad Lions', 'Make Your Destiny', False, 'swole patrol', 'Oceanus Gaming', 1, 'GoldenGitte', 'Helsinki REDS', 1, 'OFFSET Esports', 'AGF Esport', True, 'team varanid', 'Team Sincere', False, 'LiViD Gaming', 'RBG Esports', True, 'Lucid Dream', 'boom', False, 'team1', 'Rap Gang', 1, 'Big Frames', 'ex-Variance Gaming', 1, 'AVEZ', 'wine and rain', True, 'SJ Gaming', 'Planetkey Dynamics', 1, 'Vitality Bee', 'Origen BCN', False, 'Excel Esports', 'asus rog elite', True, 'MenaceGG', 'Feniks eSports', 1, 'MAD Lions', 'eXploit Esports', True, 'Sharks Esports', 'Rugratz', True, 'eUnited', 'Sharks Esports',True, 'EsportsAdviser', 'visomvet', True, 'Lilmix','Wind and Rain Sweden', 1, 'eXploit Esports', 'pRoNetAgEm', True, 'Sons of Lennart', 'Granit Gaming', True, 'team1', 'Party Astronouts', True, 'New Identity', 'Party Astronouts', 1, 'Clem', 'uThermal', False, 'Rogue', 'DnS', True, 'Revenge Gaming', 'Team Sincere', False, 'Cure', 'Nice', True, 'RagnaroK', 'MaNa', True]

for x, y, z in zip(results[::3], results[1::3], results[2::3]):
    if str(z) == 'True' or 'False':
        clean_df.loc[(clean_df['team_one'] == x) & (clean_df['team_two'] == y), 'team_one_win'] = z



In [256]:
clean_df = clean_df.dropna()
clean_df['team_one_win'] = clean_df['team_one_win'].astype('bool')
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375 entries, 0 to 445
Data columns (total 11 columns):
Game            375 non-null object
team_one        375 non-null object
team_two        375 non-null object
company         375 non-null object
team_one_odd    375 non-null float64
team_two_odd    375 non-null float64
time            375 non-null float64
margin          375 non-null float64
true_one_odd    375 non-null float64
true_two_odd    375 non-null float64
team_one_win    375 non-null bool
dtypes: bool(1), float64(6), object(4)
memory usage: 32.6+ KB


In [291]:
clean_df['true_one_per'] = 1 / clean_df['true_one_odd']
clean_df['imp_per'] = 1 / clean_df['team_one_odd']
clean_df[clean_df['Game'] == 'CSGO'].groupby('company')['team_one_win', 'true_one_per'].mean()

,team_one_win,true_one_per
company,,
10bet,0.758621,0.577339
ArcaneBet,0.740741,0.596663
Bethard,0.758621,0.577339
Betway,0.785714,0.596871
Bovada,0.750000,0.444341
Buff_bet,0.740741,0.596456
EGB,0.818182,0.565098
GG_BET,0.756757,0.577729
LOOT_BET,0.777778,0.590259


In [290]:
from scipy import stats
p_df = pd.DataFrame(columns = ['company', 'p', 't'])
for com in clean_df['company'].unique():
    t, p = stats.ttest_ind(clean_df[clean_df['company'] == com]['true_one_per'], clean_df[clean_df['company'] == com]['team_one_win'])
    p_df = p_df.append(pd.Series([com, p, t], index=p_df.columns), ignore_index = True)
    
p_df[p_df['company'] != 'Bovada'].sort_values(by=['p'], ascending = False)
    

,company,p,t
4,ArcaneBet,0.169748,-1.386532
6,Buff_bet,0.169392,-1.387702
0,LOOT_BET,0.061924,-1.886795
8,10bet,0.055645,-1.948206
9,Bethard,0.055645,-1.948206
1,Rivalry,0.051836,-1.985390
3,GG_BET,0.040920,-2.073511
7,Unikrn,0.031825,-2.208462
2,EGB,0.021640,-2.350792
5,Betway,0.019433,-2.393928
